In [4]:
!pip install efficientnet_pytorch

ERROR: Could not find a version that satisfies the requirement efficientnet_pytorch (from versions: none)
ERROR: No matching distribution found for efficientnet_pytorch


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Function to configure GPU memory growth
def configure_gpu_memory_growth():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

# Function to extract features from the image dataset using EfficientNet
def extract_features_from_folder(folder, device_name='/device:GPU:0'):
    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
        rotation_range=20,  # Data augmentation
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        shear_range=0.2,  # Additional data augmentation
        zoom_range=0.2,  # Additional data augmentation
    )
    image_generator = datagen.flow_from_directory(
        folder,
        target_size=(64, 64),
        batch_size=16,  # Increased batch size for efficient GPU utilization
        class_mode='binary',
        shuffle=False
    )
    num_samples = image_generator.samples
    num_classes = image_generator.num_classes
    steps_per_epoch = num_samples // image_generator.batch_size

    with tf.device(device_name):
        features = base_model.predict(image_generator, steps=steps_per_epoch)
    labels = image_generator.classes
    return features, labels

# Load train and test images and labels
train_folder = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/train'
test_folder = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid'

# Configure GPU memory growth
configure_gpu_memory_growth()

# Load the EfficientNet model with pre-trained weights (excluding top layer)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

X_train_features, y_train = extract_features_from_folder(train_folder, device_name='/device:GPU:0')
X_test_features, y_test = extract_features_from_folder(test_folder, device_name='/device:GPU:0')

# Define the Autoencoder
input_dim = X_train_features.shape[1] * X_train_features.shape[2] * X_train_features.shape[3]
encoding_dim = 128

ae_model = Sequential([
    Flatten(input_shape=(X_train_features.shape[1], X_train_features.shape[2], X_train_features.shape[3])),
    Dense(256, activation='relu'),  # Increased units
    BatchNormalization(),
    Dropout(0.4),  # Adjusted dropout rate
    Dense(encoding_dim, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(256, activation='relu'),  # Increased units
    BatchNormalization(),
    Dropout(0.4),
    Dense(input_dim, activation='linear'),
    Reshape((X_train_features.shape[1], X_train_features.shape[2], X_train_features.shape[3]))
])

ae_model.compile(optimizer=Adam(learning_rate=0.0001), loss=MeanSquaredError())  # Fine-tuned learning rate

# Train the Autoencoder
ae_model.fit(X_train_features, X_train_features, validation_data=(X_test_features, X_test_features), epochs=60)  # Increased epochs

# Function to predict on a specific device
def predict_on_device(model, data, device_name):
    with tf.device(device_name):
        return model.predict(data)

encoded_features_train = predict_on_device(ae_model, X_train_features, device_name='/device:GPU:0')
encoded_features_test = predict_on_device(ae_model, X_test_features, device_name='/device:GPU:0')

# Shallow CNN Model
cnn_model = Sequential([
    Flatten(input_shape=(2, 2, 1280)),
    Dense(256, activation='relu'),  # Increased units
    BatchNormalization(),
    Dropout(0.4),  # Adjusted dropout rate
    Dense(1, activation='sigmoid')
])
cnn_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])  # Fine-tuned learning rate

# Implement learning rate scheduling
initial_learning_rate = 0.0001  # Updated initial learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=2000, decay_rate=0.9, staircase=True
)
optimizer = Adam(learning_rate=lr_schedule)

# Train the Shallow CNN with the Autoencoder's encoded features
cnn_model.fit(encoded_features_train, y_train, validation_data=(encoded_features_test, y_test), epochs=120, batch_size=64, verbose=1)  # Increased epochs

# Rest of your code for evaluation and plotting
y_pred = cnn_model.predict(encoded_features_test)
y_pred = (y_pred > 0.5).astype(int)

print("Classification Report:")
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC Score: {roc_auc}")

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()


Exception: URL fetch failure on https://storage.googleapis.com/keras-applications/efficientnetb0_notop.h5: None -- [Errno -3] Temporary failure in name resolution